## SCRIPT PARA GERAÇÃO DE DADOS SOBRE NAVIOS
Este script tem por finalidade gerar dados para servirem como dataset para o modelo de classificação de Aceite de Navios

In [217]:
#IMPORTANDO BIBLIOTECAS NECESSÁRIAS
import numpy as np
import pandas as pd
import random
import datetime

In [218]:
# INTERVALOS PARA GERAÇÃO DE VALORES ALEATÓRIOS:
loa_min = 183
loa_max = 300

boca_min = 25
boca_max = 80

dwt_min = 76500
dwt_max = 160000

calado_entrada_min = 11.5
calado_entrada_max = 20

calado_saida_min = 11.5
calado_saida_max = 20

calado_aereo_min = 10
calado_aereo_max = 30

pontal_min = 5
pontal_max = 20

tamanho_lanca_min = 10
tamanho_lanca_max = 120

ano_construcao_min = datetime.date.today().year - 25
ano_construcao_max = datetime.date.today().year

situacao = 0 # 0 = aceito / 1 = recusado

tipos_navio = ['GS', 'CG', 'GSM', 'GL', 'GLP', 'CNTR']

In [219]:
# NUMERO DE AMOSTRAS QUE SERÃO GERADAS
num_amostras = 20000

# GERAÇÃO DE VALORES ALEATÓRIOS VIA NUMPY
np.random.seed(0)
random.seed(0)

nomes = np.array(['Navio' + str(i) for i in range(1, num_amostras + 1)])

loa = np.random.randint(loa_min, loa_max+1, num_amostras)

boca = np.random.randint(boca_min, boca_max+1, num_amostras)

dwt = np.random.randint(dwt_min, dwt_max+1, num_amostras)

calado_entrada = np.random.uniform(calado_entrada_min, calado_entrada_max+0.01, num_amostras)

calado_saida = np.random.uniform(calado_saida_min, calado_saida_max+0.01, num_amostras)

calado_aereo = np.random.randint(calado_aereo_min, calado_aereo_max, num_amostras)

pontal = np.random.randint(pontal_min, pontal_max, num_amostras)

tamanho_lanca = np.random.randint(tamanho_lanca_min, tamanho_lanca_max, num_amostras)

ano_construcao = np.random.randint(ano_construcao_min, ano_construcao_max+1, num_amostras)

ultimo_porto = np.array(['Porto' + str(i) for i in np.random.randint(1, 200+1, num_amostras)])

proximo_porto = np.array(['Porto' + str(i) for i in np.random.randint(1, 200+1, num_amostras)])

tipo_navio = [random.choice(tipos_navio) for _ in range(num_amostras)]

In [220]:
#CRIAÇÃO DE DATAFRAME COM VALORES
data = {
    'Nome': nomes,
    'LOA (m)': loa,
    'Boca (m)': boca,
    'DWT (ton)': dwt,
    'Calado de Entrada (m)': calado_entrada,
    'Calado de Saída (m)': calado_saida,
    'Calado Aéreo (m)': calado_aereo,
    'Pontal (m)': pontal,
    'Tamanho de Lança (m)': tamanho_lanca,
    'Ano de Construção': ano_construcao,
    'Tipo do Navio': tipo_navio,
    'Último Porto': ultimo_porto,
    'Próximo Porto': proximo_porto,
}

df = pd.DataFrame(data)
df.tail()

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto
19995,Navio19996,267,34,98699,14.385802,15.777116,11,18,16,2023,GLP,Porto26,Porto51
19996,Navio19997,284,68,132543,12.133894,12.494487,29,14,53,2001,CNTR,Porto61,Porto77
19997,Navio19998,227,34,84565,19.757311,12.505513,25,10,95,2001,GSM,Porto50,Porto160
19998,Navio19999,210,74,111345,12.458034,16.089986,20,6,103,2001,GLP,Porto135,Porto11
19999,Navio20000,193,71,78194,16.165487,19.002158,10,18,116,1999,GLP,Porto112,Porto170


In [221]:
# FUNÇÃO PARA DEFINIR RÓTULO DE CLASSE ACEITO/RECUSADO
def definir_situacao(df):
    cond_loa = df['LOA (m)'] <= 225
    cond_boca = df['Boca (m)'] <= 50
    cond_dwt = df['DWT (ton)'] <= 155000
    cond_ce = df['Calado de Entrada (m)'] <= 18.5
    cond_cs = df['Calado de Saída (m)'] <= 18.5
    cond_idade = df['Ano de Construção'] >= 2005  
    
    if cond_loa and cond_boca and cond_dwt and cond_ce and cond_cs and cond_idade:
        return 0 #aceito
    else:
        return 1#recusado

In [222]:
# CRIANDO COLUNA DA CLASSE NO DATAFRAME
df['Situação'] = df.apply(definir_situacao, axis=1)

In [223]:
# VERIFICAÇÃO DO BALANCEAMENTO DA CLASSE
df['Situação'].value_counts() #Tendencia a criação de mais recusados do que aceitos, criando necessidade de balanceamento

Situação
1    18504
0     1496
Name: count, dtype: int64

In [224]:
df.nunique()

Nome                     20000
LOA (m)                    118
Boca (m)                    56
DWT (ton)                17831
Calado de Entrada (m)    20000
Calado de Saída (m)      20000
Calado Aéreo (m)            20
Pontal (m)                  15
Tamanho de Lança (m)       110
Ano de Construção           26
Tipo do Navio                6
Último Porto               200
Próximo Porto              200
Situação                     2
dtype: int64

In [225]:
# CRIAÇÃO DE INSTÂNCIAS SEPARADAS DE DUAS AMOSTRAS (TREINO / TESTAGEM)
df_aceitos_treino = df[df['Situação'] == 0].head(500).copy()
df_recusados_treino = df[df['Situação'] == 1].head(500).copy()
df_aceitos_testagem = df[df['Situação'] == 0].tail(800).copy()
df_recusados_testagem = df[df['Situação'] == 1].tail(200).copy()

In [227]:
# CRIAÇÃO DE DATAFRAMES BALANCEADOS E DESBALANCEADOS
df_balanceado = pd.concat([df_aceitos_treino, df_recusados_treino], ignore_index=True)#Juntando aceitos e recusados num DF para treino
df_balanceado = df_balanceado.sample(frac=1).reset_index(drop=True) #Embaralhando registros

df_desbalanceado = pd.concat([df_aceitos_testagem, df_recusados_testagem], ignore_index=True) #Juntando aceitos e recusados num DF para teste
df_desbalanceado = df_desbalanceado.sample(frac=1).reset_index(drop=True) #Embaralhando registros

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto,Situação
1,Navio4787,223,30,141075,13.759191,16.606148,11,7,34,2014,GS,Porto192,Porto23,0
2,Navio120,252,37,113091,13.241776,15.010604,21,16,57,2005,CG,Porto8,Porto106,1
4,Navio5271,207,46,90295,11.636857,13.226276,26,11,15,2010,CG,Porto145,Porto170,0
5,Navio1444,205,32,154287,12.024808,12.743322,19,18,85,2016,GLP,Porto105,Porto92,0
6,Navio489,191,28,83060,19.977190,12.520758,20,18,94,2005,GL,Porto61,Porto147,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,Navio260,259,26,158882,18.930220,17.672558,29,10,53,2004,GSM,Porto172,Porto101,1
995,Navio2813,184,25,131730,17.546870,17.913528,19,8,21,2021,CG,Porto171,Porto137,0
996,Navio1226,209,34,117906,17.990231,11.737602,19,8,22,2006,GSM,Porto180,Porto96,0
997,Navio244,237,35,80971,14.481162,12.888098,16,8,110,2010,GLP,Porto139,Porto108,1


In [228]:
df_balanceado.to_csv("dados_treino1.csv")
df_desbalanceado.to_csv("dados_testagem1.csv")